In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score
from tabulate import tabulate

In [2]:
df = pd.read_csv("rice data.csv")

In [3]:
df.head()

,AREA,PERIMETER,MAJOR_AXIS,MINOR_AXIS,ECCENTRICITY,EQDIASQ,SOLIDITY,CONVEX_AREA,EXTENT,ASPECT_RATIO,...,ALLdaub4L,ALLdaub4a,ALLdaub4b,ALLdaub4Y,ALLdaub4Cb,ALLdaub4Cr,ALLdaub4XX,ALLdaub4YY,ALLdaub4ZZ,CLASS
0,7805,437.915,209.8215,48.0221,0.9735,99.6877,0.9775,7985,0.3547,4.3693,...,113.9924,65.0610,59.5989,104.8552,67.8779,63.0828,0.3673,0.3793,0.4733,Basmati
1,7503,340.757,138.3361,69.8417,0.8632,97.7400,0.9660,7767,0.6637,1.9807,...,105.7055,64.3685,62.2084,96.8375,65.5371,63.5832,0.3014,0.3144,0.3641,Arborio
2,5124,314.617,141.9803,46.5784,0.9447,80.7718,0.9721,5271,0.4760,3.0482,...,109.7155,62.6423,58.7439,100.2352,68.9753,59.8342,0.3233,0.3445,0.4448,Jasmine
3,7990,437.085,201.4386,51.2245,0.9671,100.8622,0.9659,8272,0.6274,3.9325,...,116.5405,64.9069,60.2562,107.2560,67.3298,63.2237,0.3880,0.4020,0.4904,Basmati
4,7433,342.893,140.3350,68.3927,0.8732,97.2830,0.9831,7561,0.6006,2.0519,...,107.7502,64.7071,61.3549,98.8704,66.2048,63.5378,0.3184,0.3303,0.3928,Arborio


In [4]:
df1 = df.iloc[:,16:106]
df1["CLASS"] = df["CLASS"]
df = df1

In [5]:
df.shape

(75000, 91)

In [6]:
df["CLASS"].value_counts()

CLASS
Basmati      15000
Arborio      15000
Jasmine      15000
Ipsala       15000
Karacadag    15000
Name: count, dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75000 entries, 0 to 74999
Data columns (total 91 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   meanRR      75000 non-null  float64
 1   meanRG      75000 non-null  float64
 2   meanRB      75000 non-null  float64
 3   StdDevRR    75000 non-null  float64
 4   StdDevRG    75000 non-null  float64
 5   StdDevRB    75000 non-null  float64
 6   skewRR      75000 non-null  float64
 7   skewRG      75000 non-null  float64
 8   skewRB      75000 non-null  float64
 9   kurtosisRR  75000 non-null  float64
 10  kurtosisRG  75000 non-null  float64
 11  kurtosisRB  75000 non-null  float64
 12  entropyRR   75000 non-null  int64  
 13  entropyRG   75000 non-null  int64  
 14  entropyRB   75000 non-null  int64  
 15  meanH       75000 non-null  float64
 16  meanS       75000 non-null  float64
 17  meanV       75000 non-null  float64
 18  StdDevH     75000 non-null  float64
 19  StdDevS     75000 non-nul

In [8]:
df.describe()

,meanRR,meanRG,meanRB,StdDevRR,StdDevRG,StdDevRB,skewRR,skewRG,skewRB,kurtosisRR,...,ALLdaub4V,ALLdaub4L,ALLdaub4a,ALLdaub4b,ALLdaub4Y,ALLdaub4Cb,ALLdaub4Cr,ALLdaub4XX,ALLdaub4YY,ALLdaub4ZZ
count,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,...,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000
mean,216.398005,218.205782,227.918353,15.342766,15.449838,15.477779,-1.778549,-1.938456,-2.360081,11.955533,...,0.448960,111.088252,64.379443,61.461457,101.925425,66.240541,63.202088,0.341944,0.357058,0.421176
std,13.308330,13.646445,10.682523,3.454178,3.562578,3.468618,0.948735,1.111904,0.950987,7.479528,...,0.021736,5.904854,1.175616,2.435635,5.436861,2.159109,1.174976,0.041921,0.047139,0.043137
min,153.800000,157.249900,160.158400,6.817100,6.411700,6.417500,-6.938800,-7.911800,-6.938200,1.841300,...,0.313900,82.300600,59.137900,53.653800,75.191800,58.323800,57.363400,0.159700,0.169000,0.191800
25%,206.605125,207.848625,220.927750,12.579400,12.741500,13.050675,-2.360500,-2.522300,-3.002300,6.481900,...,0.434200,106.632900,63.883800,59.465575,97.834400,64.842000,63.052800,0.309900,0.320900,0.391200
50%,215.118800,217.137550,228.801250,15.542900,15.686150,15.539300,-1.608600,-1.683350,-2.321100,9.727700,...,0.451600,110.770700,64.419350,61.424400,101.683700,66.291600,63.522050,0.340100,0.353300,0.424200
75%,225.016125,227.339300,236.171600,17.881000,18.032225,17.891800,-1.095000,-1.136700,-1.609700,15.068550,...,0.466100,115.065075,65.174200,63.076825,105.592450,68.011800,63.734000,0.370300,0.387900,0.454700
max,252.183700,252.323100,252.108500,29.967400,30.765400,30.858000,0.917900,0.771900,1.162400,75.201600,...,0.495100,126.265100,67.459000,70.284000,116.287300,73.424700,66.539100,0.463900,0.488600,0.530200


In [9]:
df = df.fillna(0)

In [10]:
print(sum(df.isnull().sum()))

0


In [11]:
df

,meanRR,meanRG,meanRB,StdDevRR,StdDevRG,StdDevRB,skewRR,skewRG,skewRB,kurtosisRR,...,ALLdaub4L,ALLdaub4a,ALLdaub4b,ALLdaub4Y,ALLdaub4Cb,ALLdaub4Cr,ALLdaub4XX,ALLdaub4YY,ALLdaub4ZZ,CLASS
0,222.9805,223.9872,241.4758,16.2950,16.4354,13.6272,-0.7986,-0.8407,-3.7377,7.1706,...,113.9924,65.0610,59.5989,104.8552,67.8779,63.0828,0.3673,0.3793,0.4733,Basmati
1,206.0380,206.2412,213.3809,18.3863,18.5343,18.9969,-0.7536,-0.7372,-0.8217,4.4926,...,105.7055,64.3685,62.2084,96.8375,65.5371,63.5832,0.3014,0.3144,0.3641,Arborio
2,201.8228,217.6475,235.0057,13.7392,15.3239,16.0249,-2.2606,-2.6764,-2.8690,12.4329,...,109.7155,62.6423,58.7439,100.2352,68.9753,59.8342,0.3233,0.3445,0.4448,Jasmine
3,228.8978,229.7151,244.6294,18.3915,18.9141,16.7398,-1.5281,-1.4967,-3.5705,8.1541,...,116.5405,64.9069,60.2562,107.2560,67.3298,63.2237,0.3880,0.4020,0.4904,Basmati
4,210.4471,210.2988,220.8827,19.6370,19.6635,20.0206,-1.0165,-1.0176,-1.1514,4.6467,...,107.7502,64.7071,61.3549,98.8704,66.2048,63.5378,0.3184,0.3303,0.3928,Arborio
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74995,204.4210,201.7319,208.9247,13.8755,13.8819,14.0564,-0.4432,-0.3778,-0.4530,6.9004,...,103.9529,64.9225,62.4355,95.2780,65.5114,64.4457,0.2895,0.2997,0.3455,Arborio
74996,213.9765,212.5912,229.1017,23.9463,24.2106,25.4296,-0.8286,-0.7967,-1.1883,3.8488,...,108.9778,65.4571,59.9502,100.2301,67.5089,63.6028,0.3335,0.3426,0.4257,Karacadag
74997,204.7994,207.2316,216.9780,16.1357,16.4876,16.4977,-1.2066,-1.2433,-1.3866,5.9842,...,106.0881,64.1869,61.3876,97.1585,66.2445,63.0596,0.3028,0.3164,0.3761,Arborio
74998,234.8254,236.9970,233.0475,9.9312,9.3609,9.0690,-2.4975,-3.4904,-3.2237,21.9007,...,119.2037,63.3545,64.8200,109.3027,63.3122,63.5967,0.3970,0.4215,0.4469,Ipsala


In [12]:
df = np.array(df)
df = pd.DataFrame(df)
X = df.drop(columns = [90])
y = df[90]

sc = StandardScaler()
X = sc.fit_transform(X)
X = pd.DataFrame(X)

In [13]:
kf = KFold(n_splits=10, shuffle=False)

models = {
    "MLP": MLPClassifier(hidden_layer_sizes=(64, 32, 16, 8), activation='logistic', max_iter=10000, random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "SVM": SVC(kernel='linear', random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=10),
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42)
}

model_metrics = {name: {"Accuracy": [], "Error": [], "Precision": [], "Recall": [], "F1-Score": [], "Kappa": []} for name in models.keys()}

for fold, (train_idx, test_idx) in enumerate(kf.split(df)):
    print(f"Fold {fold + 1} :")
    
    X_train, X_test = X.iloc[train_idx, :], X.iloc[test_idx, :]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        acc = accuracy_score(y_test, y_pred)
        error = 1 - acc  # Error rate
        prec = precision_score(y_test, y_pred, average="weighted", zero_division=0)
        rec = recall_score(y_test, y_pred, average="weighted")
        f1 = f1_score(y_test, y_pred, average="weighted")
        kappa = cohen_kappa_score(y_test, y_pred)

        model_metrics[name]["Accuracy"].append(acc)
        model_metrics[name]["Error"].append(error)
        model_metrics[name]["Precision"].append(prec)
        model_metrics[name]["Recall"].append(rec)
        model_metrics[name]["F1-Score"].append(f1)
        model_metrics[name]["Kappa"].append(kappa)

        print(f"{name}: Acc={acc:.4f}, Err={error:.4f}, Prec={prec:.4f}, Rec={rec:.4f}, F1={f1:.4f}, Kappa={kappa:.4f}")

    print("-" * 50)

table_data = [
    [name, 
     f"{np.mean(metrics['Accuracy']):.4f}", 
     f"{np.mean(metrics['Error']):.4f}", 
     f"{np.mean(metrics['Precision']):.4f}", 
     f"{np.mean(metrics['Recall']):.4f}", 
     f"{np.mean(metrics['F1-Score']):.4f}", 
     f"{np.mean(metrics['Kappa']):.4f}"]
    for name, metrics in model_metrics.items()
]

print(tabulate(table_data, headers=["Model", "Avg Acc", "Avg Error", "Avg Prec", "Avg Recall", "Avg F1", "Avg Kappa"], tablefmt="grid"))

Fold 1 :
MLP: Acc=0.9920, Err=0.0080, Prec=0.9920, Rec=0.9920, F1=0.9920, Kappa=0.9900
Random Forest: Acc=0.9873, Err=0.0127, Prec=0.9873, Rec=0.9873, F1=0.9873, Kappa=0.9842
SVM: Acc=0.9919, Err=0.0081, Prec=0.9919, Rec=0.9919, F1=0.9919, Kappa=0.9898
KNN: Acc=0.9844, Err=0.0156, Prec=0.9844, Rec=0.9844, F1=0.9844, Kappa=0.9805
Logistic Regression: Acc=0.9909, Err=0.0091, Prec=0.9909, Rec=0.9909, F1=0.9909, Kappa=0.9887
Decision Tree: Acc=0.9740, Err=0.0260, Prec=0.9740, Rec=0.9740, F1=0.9740, Kappa=0.9675
--------------------------------------------------
Fold 2 :
MLP: Acc=0.9939, Err=0.0061, Prec=0.9939, Rec=0.9939, F1=0.9939, Kappa=0.9923
Random Forest: Acc=0.9904, Err=0.0096, Prec=0.9904, Rec=0.9904, F1=0.9904, Kappa=0.9880
SVM: Acc=0.9924, Err=0.0076, Prec=0.9924, Rec=0.9924, F1=0.9924, Kappa=0.9905
KNN: Acc=0.9857, Err=0.0143, Prec=0.9858, Rec=0.9857, F1=0.9857, Kappa=0.9822
Logistic Regression: Acc=0.9905, Err=0.0095, Prec=0.9905, Rec=0.9905, F1=0.9905, Kappa=0.9882
Decision Tr

In [19]:
from sklearn.metrics import confusion_matrix

labels = np.unique(y)
conf_matrix_sum = {name: np.zeros((len(labels), len(labels))) for name in models.keys()}

for fold, (train_idx, test_idx) in enumerate(kf.split(df)):
    print(f"Fold {fold + 1} :")
    X_train, X_test = X.iloc[train_idx, :], X.iloc[test_idx, :]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        cm = confusion_matrix(y_test, y_pred, labels=labels)
        conf_matrix_sum[name] += cm
        print(f"{name}")

    print(f"Fold {fold + 1} Completed")
    print("-" * 50)

conf_matrix_total = {name: cm.astype(int) for name, cm in conf_matrix_sum.items()}

conf_matrix_data = []
for i, actual in enumerate(labels):
    for name, cm in conf_matrix_total.items():
        row = [actual, name]
        row.extend(cm[i])
        conf_matrix_data.append(row)

columns = ["Actual Class", "Algorithm"] + list(labels)
df_conf_matrix = pd.DataFrame(conf_matrix_data, columns=columns)

print(df_conf_matrix)

Fold 1 :
MLP
Random Forest
SVM
KNN
Logistic Regression
Decision Tree
Fold 1 Completed
--------------------------------------------------
Fold 2 :
MLP
Random Forest
SVM
KNN
Logistic Regression
Decision Tree
Fold 2 Completed
--------------------------------------------------
Fold 3 :
MLP
Random Forest
SVM
KNN
Logistic Regression
Decision Tree
Fold 3 Completed
--------------------------------------------------
Fold 4 :
MLP
Random Forest
SVM
KNN
Logistic Regression
Decision Tree
Fold 4 Completed
--------------------------------------------------
Fold 5 :
MLP
Random Forest
SVM
KNN
Logistic Regression
Decision Tree
Fold 5 Completed
--------------------------------------------------
Fold 6 :
MLP
Random Forest
SVM
KNN
Logistic Regression
Decision Tree
Fold 6 Completed
--------------------------------------------------
Fold 7 :
MLP
Random Forest
SVM
KNN
Logistic Regression
Decision Tree
Fold 7 Completed
--------------------------------------------------
Fold 8 :
MLP
Random Forest
SVM
KNN
Logist

In [20]:
print(tabulate(df_conf_matrix, headers='keys', tablefmt='grid'))

+----+----------------+---------------------+-----------+-----------+----------+-----------+-------------+
|    | Actual Class   | Algorithm           |   Arborio |   Basmati |   Ipsala |   Jasmine |   Karacadag |
+====+================+=====================+===========+===========+==========+===========+=============+
|  0 | Arborio        | MLP                 |     14829 |         9 |        0 |       143 |          19 |
+----+----------------+---------------------+-----------+-----------+----------+-----------+-------------+
|  1 | Arborio        | Random Forest       |     14780 |         5 |        0 |       198 |          17 |
+----+----------------+---------------------+-----------+-----------+----------+-----------+-------------+
|  2 | Arborio        | SVM                 |     14811 |        10 |        0 |       172 |           7 |
+----+----------------+---------------------+-----------+-----------+----------+-----------+-------------+
|  3 | Arborio        | KNN          